In [1]:
import os

In [27]:
from collections import defaultdict 

targetDir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder3'
lstDir = os.listdir(targetDir)
sizeDict = defaultdict(int)
for i in lstDir:
    size = float(i[-17:-5])
    sizeRange = int(size*2)/2
    sizeDict[sizeRange] += 1

print(sizeDict)
print(dict(sorted(sizeDict.items())))
print(dict(sorted(sizeDict.items(), key=lambda item: item[1])))




defaultdict(<class 'int'>, {7.0: 577, 4.0: 173, 3.0: 320, 2.5: 524, 3.5: 154, 6.0: 275, 6.5: 505, 5.5: 326, 1.0: 918, 5.0: 304, 9.0: 641, 12.0: 506, 13.5: 513, 14.5: 460, 15.0: 495, 14.0: 517, 1.5: 176, 13.0: 518, 8.0: 815, 12.5: 512, 11.5: 577, 11.0: 669, 10.5: 754, 10.0: 673, 9.5: 644, 2.0: 541, 8.5: 696, 7.5: 672, 4.5: 160, 22.0: 291, 21.5: 213, 21.0: 262, 20.5: 229, 20.0: 250, 15.5: 468, 19.5: 250, 19.0: 249, 18.5: 343, 18.0: 315, 17.5: 378, 17.0: 393, 16.0: 448, 16.5: 450, 22.5: 226, 23.0: 256, 23.5: 146, 24.0: 117, 24.5: 111, 25.0: 108, 28.5: 71, 28.0: 64, 29.0: 56, 29.5: 88, 30.0: 43, 30.5: 55, 31.0: 37, 31.5: 54, 32.0: 47, 32.5: 40, 33.0: 50, 27.5: 70, 27.0: 68, 26.5: 96, 26.0: 85, 25.5: 86, 33.5: 116, 34.0: 50, 0.5: 304})
{0.5: 304, 1.0: 918, 1.5: 176, 2.0: 541, 2.5: 524, 3.0: 320, 3.5: 154, 4.0: 173, 4.5: 160, 5.0: 304, 5.5: 326, 6.0: 275, 6.5: 505, 7.0: 577, 7.5: 672, 8.0: 815, 8.5: 696, 9.0: 641, 9.5: 644, 10.0: 673, 10.5: 754, 11.0: 669, 11.5: 577, 12.0: 506, 12.5: 512, 13

5.0
